In [0]:
"""
El presente algoritmo demuestra el concepto de equidad a corto y largo plazo. Utiliza una combinacion de factores para construir una clasificacion de valores liquidos en un universo negociable. Luego pasa un tiempo en los valores mejor clasificados en en corto plazo y en los valores mas bajos.

"""

import quantopian.algorithm as algo
import quantopian.optimize as opt
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import SimpleMovingAverage

from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.experimental import risk_loading_pipeline

from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data import Fundamentals

# Parametros de restriccion 
MAX_GROSS_LEVERAGE = 1.0
TOTAL_POSITIONS = 600

# Se define el tamaño maximo de posicion que se puede mantener para cualquier
# stock dado. 
MAX_SHORT_POSITION_SIZE = 2.0 / TOTAL_POSITIONS
MAX_LONG_POSITION_SIZE = 2.0 / TOTAL_POSITIONS


def initialize(context):
    """
    Una funcion central llamada de manera automatica una vez al comienzo de un         backtest.

    Parametros
    ----------
    context : AlgorithmContext
        Un objeto que se puede usar para almacenar el estado en el que se desea           mantener en el algoritmo. context se pasa automaticamente para                     inicializar, before_trading_start, handle_data, y todas las funciones se            ejecutan a traves de schedule_function.
        
        context proporciona el atributo de cartera, que se puede utilizar para             recuperar informacion.
    """
   
    algo.attach_pipeline(make_pipeline(), 'long_short_equity_template')

    # Adjunte el pipeline para los factores de riesgo que deseamos
    # neutralizar en el paso de la optimizacion. La cadena de 'risk_factors' 
    # es utilizado para recuperar la salidad del pipeline en before_trading_start
    algo.attach_pipeline(risk_loading_pipeline(), 'risk_factors')

    # Programando la funcion de reequilibrio
    algo.schedule_function(func=rebalance,
                           date_rule=algo.date_rules.week_start(),
                           time_rule=algo.time_rules.market_open(hours=0, minutes=30),
                           half_days=True)

    # Registro de las variables de cartera al final del dia
    algo.schedule_function(func=record_vars,
                           date_rule=algo.date_rules.every_day(),
                           time_rule=algo.time_rules.market_close(),
                           half_days=True)


def make_pipeline():
    """
    Funcion que crea y devuelve nuestros pipeline.
    
    Returns
    -------
    pipe : Pipeline
        Representa el calculo que nos gustara realizar en los activos,
    """
    # Los factores que se crean se basan en datos fundamentales y un movimiento
    # promedio de datos de opinion
    
    value = Fundamentals.ebit.latest /             Fundamentals.enterprise_value.latest
    quality = Fundamentals.roe.latest
    sentiment_score = SimpleMovingAverage(
        inputs=[stocktwits.bull_minus_bear],
        window_length=3,
    )
   
    #  Variables que usaremos en el presente ejercicio

    cap_mercado = Fundamentals.market_cap.latest
    cash_flow = Fundamentals.cfo_per_share.latest
    gr_loan = Fundamentals.gross_loan.latest
    income_tax = Fundamentals.income_tax_payable.latest
    growth = Fundamentals.dps_growth.latest
    universe = QTradableStocksUS()
    cashr = Fundamentals.cash_return.latest
   
    # Se excluyen los datos atipicos de las varibles con el metodo winsorizing
    
    value_winsorized = value.winsorize(min_percentile=0.05, max_percentile=0.95)
    quality_winsorized = quality.winsorize(min_percentile=0.05, max_percentile=0.95)
    sentiment_score_winsorized = sentiment_score.winsorize(min_percentile=0.05,                   max_percentile=0.95)
   
    cap_mercado_winsorized = cap_mercado.winsorize(min_percentile=0.05,    max_percentile=0.95)
    cash_flow_winsorized = cash_flow.winsorize(min_percentile=0.05,    max_percentile=0.95)
   
    gr_loan_winsorized = gr_loan.winsorize(min_percentile=0.05,    max_percentile=0.95)
   
    income_tax_winsorized = income_tax.winsorize(min_percentile=0.05,    max_percentile=0.95)
   
    growth_winsorized = growth.winsorize(min_percentile=0.05,    max_percentile=0.95)
   
    cashr_winsorized = cashr.winsorize(min_percentile=0.05,    max_percentile=0.95)
   

    # Configuracion de pesos en terminos de profit 
    
    value_weight = 0.14
    quality_weight = 0.14
    sentiment_score_weight = 0.14
    cap_mercado_weight = 0.3
    cash_flow_weight = 0.16
    gr_loan_weight = 0.14
    income_tax_weight = 0.14
    growth_weight = 0.2
    cashr_weight = 0.12
    
   
    # Factor combinado con los respectivos pesos, usando la funcion zscore para 
    # normalizar los datos
    
    combined_factor = (
        value_winsorized.zscore() * value_weight +
        quality_winsorized.zscore() * quality_weight +
        sentiment_score_winsorized.zscore() * sentiment_score_weight +
        cap_mercado_winsorized.zscore() * cap_mercado_weight +
        cash_flow_winsorized.zscore() * cash_flow_weight +
        gr_loan_winsorized.zscore() * gr_loan_weight +
        income_tax_winsorized.zscore() * income_tax_weight +
        growth_winsorized.zscore() * growth_weight +
        cashr_winsorized.zscore() * cashr_weight
    )

    # 
    longs = combined_factor.top(TOTAL_POSITIONS//2, mask=universe)
    shorts = combined_factor.bottom(TOTAL_POSITIONS//2, mask=universe)

    # La salida final del pipeline solo deberia incluir
    # las 300 acciones superiores/inferiores.
    long_short_screen = (longs | shorts)

    # Create pipeline
    pipe = Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts,
            'combined_factor': combined_factor
        },
        screen=long_short_screen
    )
    return pipe


def before_trading_start(context, data):
    """
    Funcion principal opcional llamada de manera automatica antes de la apertura del mercado
    
    """
    # Llame a algo.pipeline_output para obtener la salida
    
    context.pipeline_data = algo.pipeline_output('long_short_equity_template')

    # Este dataframe contendra todas nuestras cargas de riesgo
    context.risk_loadings = algo.pipeline_output('risk_factors')


def record_vars(context, data):
    """
  Una funcion programada para ejecutar todos los dias al cierre del mercado para registrar

    """
    # PTrace el numero de posiciones a lo largo del tiempo
    algo.record(num_positions=len(context.portfolio.positions))


# Llamado al comienzo de cada mes para reequilibrar
# La lista de largos y cortos
def rebalance(context, data):
    """
    Funcion programada para ejectarse una vez cada  lunes a las 10 am. Reequilibrar las listas de largos y cortos.

    
    """
    # Recuperar la salida de pipeline 
    pipeline_data = context.pipeline_data

    risk_loadings = context.risk_loadings

    # Se define el objetivo de la optimize API 
    # Seleccionado MaximizeAlpha por que creemos en el factor combinado
    # Esta rutina optimizara el retorno esperado de nuestro algortimo.
    objective = opt.MaximizeAlpha(pipeline_data.combined_factor)

    # Definir la lista de restricciones
    constraints = []
    # Restringir nuestro apalancamiento bruto maximo
    constraints.append(opt.MaxGrossExposure(MAX_GROSS_LEVERAGE))

    # Requerir que nuestro algoritmo permanezca neutral en dolares
    constraints.append(opt.DollarNeutral())

    # Agregar la restriccion RiskModelExposure para hacer uso de
    # restriscciones de modelo de riesgo predeterminadas.
    neutralize_risk_factors = opt.experimental.RiskModelExposure(
        risk_model_loadings=risk_loadings,
        version=0
    )
    constraints.append(neutralize_risk_factors)

    # Con esta restriccion, exigimos que ninguna posicion pueda compensar
    # mayor que MAX_SHORT_POSITION_SIZE en el lado corto
    # no mayor que MAX_LONG_POSITION_SIZE en el lado largo
    constraints.append(
        opt.PositionConcentration.with_equal_bounds(
            min=-MAX_SHORT_POSITION_SIZE,
            max=MAX_LONG_POSITION_SIZE
        ))

    # Junta todas las piezas que definimos anteriormente pasado
    # en la funcion algo.order_optimal_portfolio function. Esto maneja
    # toda nuestra locica de pedido asignando pesos apropiados.
    algo.order_optimal_portfolio(
        objective=objective,
        constraints=constraints
    )